In [1]:
#!pip install ipynb --upgrade

In [2]:
#!pip install --upgrade pip

In [3]:
#!pip install keras-tcn
#!pip install tensorflow
#!pip install ipynb
#!pip install plotly

In [4]:
import pickle
import time
import os
import yaml
import sys
import pandas as pd

sys.path.append('C:/Users/Alex-/Desktop/Beslutsstödssystem/models-decision-system')
os.chdir("..")
import Decision
import Models

os.chdir("MockPipeline")

os.chdir("../../pipeline")
import ipynb.fs.full.processing as processing
import ipynb.fs.full.features as features
import ipynb.fs.full.training as training
import ipynb.fs.full.analysis as analysis
import ipynb.fs.full.storage as storage
import ipynb.fs.full.visualize as visualize
os.chdir("../models-decision-system/MockPipeline")

In [5]:
import os

In [7]:
def load_yaml(path):
    with open(path, mode='r') as file:
        return yaml.load(file, Loader=yaml.FullLoader)

In [8]:
settings = load_yaml('mock_settings.yaml')

In [9]:
settings['ensemble']['models'][0]

{'linreg': {'split': {'train': 0.8, 'test': 0.2}}}

In [10]:
dataframe = processing.create_dataframe(settings)

In [11]:
dataset = features.add(dataframe, settings['features'])

In [12]:
dataset

,Close,Momentum,SD,Label
Date_Timestamp,,,,
2010-02-01,1.39200,-0.04649,14.842308,1.39737
2010-02-02,1.39737,-0.03928,13.972612,1.38969
2010-02-03,1.38969,-0.05070,11.302307,1.37395
2010-02-04,1.37395,-0.05393,11.014429,1.36763
2010-02-05,1.36763,-0.04356,11.636518,1.36409
...,...,...,...,...
2019-12-25,1.10944,0.00301,54.215370,1.11012
2019-12-26,1.11012,0.00089,50.207241,1.11713
2019-12-27,1.11713,0.00388,49.232845,1.11813


In [13]:
data_linreg = features.split(dataset, 'linreg', settings['ensemble']['models'][0]['linreg'])

In [14]:
linreg = Models.train_model(data_linreg, 'linreg', settings['ensemble']['models'][0])

In [15]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [16]:
settings['ensemble']['models'][1]['lstm']['layers']

[{'lstm': {'value': 120, 'activation': 'relu'}},
 {'dropout': {'value': 0.15}},
 {'dense': {'value': 50, 'activation': 'relu'}},
 {'dense': {'value': 1}}]

In [17]:
lstm = Models.train_model(data_lstm, 'lstm', settings['ensemble']['models'][1]['lstm'])

Epoch 1/7
233/233 [==============================] - 1s 5ms/step - loss: 0.0271 - val_loss: 0.0013
Epoch 2/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0053 - val_loss: 0.0030
Epoch 3/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0045 - val_loss: 0.0015
Epoch 4/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0033 - val_loss: 6.3325e-04
Epoch 5/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0029 - val_loss: 0.0016
Epoch 6/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0025 - val_loss: 4.7447e-04
Epoch 7/7
233/233 [==============================] - 1s 3ms/step - loss: 0.0023 - val_loss: 0.0015


In [18]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
from sklearn import preprocessing
from sklearn import utils

In [19]:
df = pd.read_csv("../MockData.csv")
df_dec = pd.read_csv("../decision.csv")

In [20]:
df_dec = df_dec.set_index("DateTime")
df = df.set_index("DateTime")

In [21]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split

In [22]:
df

,Close,RoC,S%D,PSY12,ASY5
DateTime,,,,,
2010-01-27,0.817467,0.279074,0.239967,0.272727,0.452398
2010-01-28,0.802634,0.256945,0.173515,0.272727,0.376690
2010-01-29,0.779823,0.260848,0.136936,0.272727,0.291014
2010-01-31,0.782677,0.276472,0.108876,0.363636,0.304610
2010-02-01,0.793195,0.281944,0.078999,0.363636,0.385942
...,...,...,...,...,...
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453
2019-12-27,0.175454,0.565530,0.421044,0.818182,0.586624
2019-12-29,0.177701,0.567630,0.451166,0.818182,0.585169


In [23]:
df_dec

,Forecast,Actual,f_momentum,a_momentum,Invest next day
DateTime,,,,,
2018-05-22,0.330828,0.296319,NaN,NaN,Hold
2018-05-23,0.333239,0.299195,NaN,NaN,Hold
2018-05-24,0.328026,0.282542,-0.002801,-0.013777,Sell
2018-05-25,0.322993,0.291667,-0.010246,-0.007529,Hold
2018-05-27,0.310946,0.277665,-0.017081,-0.004877,Buy
...,...,...,...,...,...
2019-12-24,0.168590,0.158172,-0.001180,0.000607,Hold
2019-12-25,0.167789,0.159700,-0.000904,0.003124,Hold
2019-12-26,0.166317,0.175454,-0.002273,0.017282,Hold


In [24]:
col = "Invest next day"
conditions = [df_dec[col] == "Buy"]
choices = ["1"]
df_dec["buy"] = np.select(conditions, choices, default="0")
conditions = [df_dec[col] == "Sell"]
choices = ["1"]
df_dec["sell"] = np.select(conditions, choices, default="0")
conditions = [df_dec[col] == "Hold"]
choices = ["1"]
df_dec["hold"] = np.select(conditions, choices, default="0")

In [25]:
df_dec

,Forecast,Actual,f_momentum,a_momentum,Invest next day,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.330828,0.296319,NaN,NaN,Hold,0,0,1
2018-05-23,0.333239,0.299195,NaN,NaN,Hold,0,0,1
2018-05-24,0.328026,0.282542,-0.002801,-0.013777,Sell,0,1,0
2018-05-25,0.322993,0.291667,-0.010246,-0.007529,Hold,0,0,1
2018-05-27,0.310946,0.277665,-0.017081,-0.004877,Buy,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.168590,0.158172,-0.001180,0.000607,Hold,0,0,1
2019-12-25,0.167789,0.159700,-0.000904,0.003124,Hold,0,0,1
2019-12-26,0.166317,0.175454,-0.002273,0.017282,Hold,0,0,1


In [26]:
df["buy"] = df_dec["buy"]
df["sell"] = df_dec["sell"]
df["hold"] = df_dec["hold"]

In [27]:
df = df.dropna()
df

,Close,RoC,S%D,PSY12,ASY5,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.314006,0.480472,0.194738,0.363636,0.479935,0,0,1
2018-05-23,0.296319,0.444085,0.198627,0.363636,0.432865,0,0,1
2018-05-24,0.299195,0.411007,0.185304,0.363636,0.462282,0,1,0
2018-05-25,0.282542,0.349541,0.151715,0.272727,0.400889,0,0,1
2018-05-27,0.291667,0.366831,0.125065,0.272727,0.414903,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.156576,0.526441,0.519878,0.727273,0.477280,0,0,1
2019-12-25,0.158172,0.504520,0.453905,0.727273,0.476746,0,0,1
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453,0,0,1


In [28]:
df.columns[:5]

Index(['Close', 'RoC', 'S%D', 'PSY12', 'ASY5'], dtype='object')

In [29]:
df

,Close,RoC,S%D,PSY12,ASY5,buy,sell,hold
DateTime,,,,,,,,
2018-05-22,0.314006,0.480472,0.194738,0.363636,0.479935,0,0,1
2018-05-23,0.296319,0.444085,0.198627,0.363636,0.432865,0,0,1
2018-05-24,0.299195,0.411007,0.185304,0.363636,0.462282,0,1,0
2018-05-25,0.282542,0.349541,0.151715,0.272727,0.400889,0,0,1
2018-05-27,0.291667,0.366831,0.125065,0.272727,0.414903,1,0,0
...,...,...,...,...,...,...,...,...
2019-12-24,0.156576,0.526441,0.519878,0.727273,0.477280,0,0,1
2019-12-25,0.158172,0.504520,0.453905,0.727273,0.476746,0,0,1
2019-12-26,0.159700,0.475611,0.395299,0.727273,0.525453,0,0,1


In [30]:
def foofoofoo(lblz):
    container = []
    for row in lblz:
        word = ''
        for value in row:
            word = word + str(value)
        container.append(word)
    return container

In [31]:
labels = df[["buy", "sell", "hold"]].to_numpy()

In [32]:
labels = foofoofoo(labels)

In [33]:
    forecast_days = 1
    #df['Label'] = df['Close'].shift(-forecast_days)
    df_dec.dropna()
    data = df
    data = data.dropna(how="any")
    x = data[df.columns[:5]].to_numpy()
    y = data[df.columns[-4]].to_numpy()
    y2 = data[df.columns[-3:]].to_numpy()

## Dataset y


In [34]:
def create_split(x, y):
    tss = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tss.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
    return x_train, y_train, x_test, y_test

In [35]:
x_train, y_train, x_test, y_test = create_split(x, y)
x_train_2, y_train_2, x_test_2, y_test_2 = create_split(x, y2)

In [36]:
y_test_2 = foofoofoo(y_test_2)
y_train_2 = foofoofoo(y_train_2)

## Dataset y2

In [37]:
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.svm import LinearSVC
#OneVsRestClassifier(LinearSVC(random_state=0)).fit(x, y).predict(x)

In [38]:
from sklearn.preprocessing import MinMaxScaler

In [39]:
scaler = MinMaxScaler(feature_range=(0,1))

x_train = scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(y_train)
print(training_scores_encoded)
print(utils.multiclass.type_of_target(y_train))
print(utils.multiclass.type_of_target(y_train.astype('int')))
print(utils.multiclass.type_of_target(training_scores_encoded))

In [95]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from pprint import pprint
from sklearn import metrics

# Gridsearch random forest

https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74


In [256]:
def select_model(settings):
    """Select a classification model type. 
    Supported models:
    Random Forest Classifier (default)"""
    if(settings["model"] == rfc):
        model = RandomForestClassifier()
    else:
        model = RandomForestClassifier()
    return model

In [257]:
def random_search(settings):
    
    # CREATE THE CORRECT MODEL TYPE
    model = select_model(settings)

    # CREATE THE RANDOMIZED SEARCH CV
    rscv = RandomizedSearchCV(estimator=model,
                                  param_distributions=settings['random_params'],
                                  n_iter=100,
                                  cv = 3,
                                  verbose=2,
                                  random_state=42,
                                  n_jobs=-1)

    # FIT THE RANDOM SEARCH
    rscv.fit(settings["x_train"], settings["y_train"])
    
    # TEST RUNS
    #base_model = RandomForestClassifier(random_state=4)   
    #base_model.fit(settings["x_train"], settings["y_train"])
    #base_accuracy = evaluate(base_model, settings["x_test"], settings["y_test"])
    best_random = rscv.best_estimator_
    random_accuracy = evaluate(best_random, settings["x_test"], settings["y_test"])
    #rscv.best_params_
    
    # RETURN THE BEST VALUES FROM THE RANDOMIZED SEARCH AS A DICTIONARY
    
    
    return rscv.best_params_, random_accuracy

In [258]:
def grid_search(settings, grid_input):
    # CREATE PARAM GRID FROM INPUT VALUES
    bootstrap = grid_input["bootstrap"]
    max_depth = grid_input["max_depth"]
    max_features = grid_input["max_features"]
    min_samples_leaf = grid_input["min_samples_leaf"]
    min_samples_split = grid_input["min_samples_split"]
    n_estimators = grid_input["n_estimators"]
    
    # SELECT MODEL
    rf = select_model(settings("model"))
    
    # CREATE GRID SEARCH CV
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
    best_grid = grid_search.best_estimator_
    grid_accuracy = evaluate(best_grid, x_test_2, y_test_2)
    
    # CREATE DICT WITH BEST PARAMS
    grid_results = {
        "bootstrap": grid_search.best_params_["bootstrap"],
        "max_depth": grid_search.best_params_["max_depth"],
        "max_features": grid_search.best_params_["max_features"],
        "min_samples_leaf": grid_search.best_params_["min_samples_leaf"],
        "min_samples_split": grid_search.best_params_["min_samples_split"],
        "n_estimators": grid_search.best_params_["n_estimators"]
    }
    return grid_results, grid_accuracy

In [259]:
def search_params(settings):
    # FETCH MODEL FROM SETTINGS
    
    # RANDOM SEARCH
    grid_input, random_score = random_search(settings)
    
    # GRID SEARCH BASED ON RANDOM SEARCH BEST PARAMS
    grid_search_results = grid_search(settings, grid_input)
    
    # RUN MODEL ON BEST PARAMS
    
    return 0

In [244]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [238]:
input_settings = {
    'model': "rfc",
    'random_params': random_grid,
    "x_train": x_train_2,
    "x_test": x_test_2,
    "y_train:": y_train_2,
    "y_test": y_train_2
}

In [239]:
search_params(input_settings)

NameError: name 'grid_input' is not defined

In [61]:
y_train_2

['001',
 '001',
 '010',
 '001',
 '100',
 '010',
 '001',
 '001',
 '100',
 '010',
 '001',
 '001',
 '001',
 '100',
 '001',
 '001',
 '001',
 '010',
 '100',
 '001',
 '010',
 '001',
 '001',
 '100',
 '010',
 '001',
 '001',
 '001',
 '100',
 '001',
 '001',
 '010',
 '001',
 '100',
 '001',
 '010',
 '001',
 '001',
 '001',
 '100',
 '001',
 '001',
 '001',
 '010',
 '001',
 '001',
 '100',
 '001',
 '001',
 '010',
 '001',
 '100',
 '001',
 '010',
 '100',
 '001',
 '010',
 '001',
 '001',
 '100',
 '001',
 '001',
 '010',
 '001',
 '001',
 '001',
 '100',
 '001',
 '001',
 '010',
 '001',
 '001',
 '001',
 '001',
 '001',
 '100',
 '010',
 '001',
 '100',
 '001',
 '010',
 '001',
 '100',
 '001',
 '001',
 '001',
 '001',
 '010',
 '001',
 '001',
 '001',
 '100',
 '001',
 '010',
 '001',
 '001',
 '001',
 '100',
 '001',
 '001',
 '010',
 '001',
 '001',
 '001',
 '100',
 '001',
 '001',
 '001',
 '001',
 '001',
 '010',
 '001',
 '100',
 '001',
 '010',
 '001',
 '001',
 '100',
 '001',
 '010',
 '001',
 '001',
 '100',
 '001',
 '010',


In [120]:

rfc = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator=rfc,
                              param_distributions=random_grid,
                              n_iter=100,
                              cv = 3,
                              verbose=2,
                              random_state=42,
                              n_jobs=-1)

rf_random.fit(x_train_2, y_train_2)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.1min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [217]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3, 'sqrt'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [1,2,3,4],
    'n_estimators': [1400, 1500, 1600, 2500]
}

In [216]:
def evaluate(model, test_features, test_labels, encode=True):
    """Evaluate a classifcation model, for example Random Forest Classification
    
    Parameters:
    model: sklearn classification model
    test_features: np.array of features
    test_labels: np.array of labels 
    encode (default=True): boolean, encodes values to int if set to true
    
    Returns:
    accuracy: float, metrics.accuracy_score
    precision: float, metrics.precision_score
    recall: float, metrics.precision_score
    confusion: 
    """
    # PREDICT
    predictions = model.predict(test_features)
    
    # OPTIONAL ENCODING
    if(encode == True):
        le = preprocessing.LabelEncoder()
        le.fit(test_labels)
        test_labels = le.transform(test_labels)
        predictions = le.transform(predictions)

    # CREATE CONFUSION MATRIX
    confusion = metrics.confusion_matrix(test_labels, predictions, labels=[0, 1, 2])
    
    # PULL ACCURACY FROM METRICS
    accuracy = metrics.accuracy_score(test_labels, predictions)
    precision = metrics.precision_score(test_labels, predictions, average='micro')
    recall = metrics.recall_score(test_labels, predictions, average='micro')
    #print('Model Performance')
    #print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    #print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    # RETURN THE CREATED SCORES
    return accuracy, precision, recall, confusion

In [203]:
base_model = RandomForestClassifier(random_state=4)
base_model.fit(x_train_2, y_train_2)
base_accuracy = evaluate(base_model, x_test_2, y_test_2)
print("==========================")
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, x_test_2, y_test_2)

In [249]:
rf_random.best_params_

{'n_estimators': 1800,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 90,
 'bootstrap': True}

In [184]:
# Create the parameter grid based on the results of random search 
# Create a based model

rf = RandomForestClassifier()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [185]:
grid_search.fit(x_train_2, y_train_2)

grid_search.best_params_

Fitting 3 folds for each of 576 candidates, totalling 1728 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   42.0s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 1728 out of 1728 | elapsed:  8.3min finished


{'bootstrap': True,
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 4,
 'n_estimators': 1600}

In [189]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, x_test_2, y_test_2)

[[63  0  0]
 [10  0  0]
 [10  0  0]]
Model Performance
Accuracy = 0.76%.


In [191]:
bootstrap = grid_search.best_params_["bootstrap"]
max_depth = grid_search.best_params_["max_depth"]
max_features = grid_search.best_params_["max_features"]
min_samples_leaf = grid_search.best_params_["min_samples_leaf"]
min_samples_split = grid_search.best_params_["min_samples_split"]
n_estimators = grid_search.best_params_["n_estimators"]

In [209]:
rfc = RandomForestClassifier(bootstrap=bootstrap, 
                             max_depth=max_depth, 
                             max_features=max_features, 
                             min_samples_leaf=min_samples_leaf,
                            min_samples_split=min_samples_split,
                            n_estimators=n_estimators)
rfc.fit(x_train_2, y_train_2)
model_output = evaluate(rfc, x_test_2, y_test_2)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [215]:
model_output[3]

array([[62,  0,  1],
       [10,  0,  0],
       [10,  0,  0]], dtype=int64)

In [ ]:
{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400}

## random forest

In [41]:
y_train

array([0.47993472, 0.43286509, 0.46228157, 0.40088851, 0.41490263,
       0.36220487, 0.35250225, 0.45311436, 0.54190878, 0.47688486,
       0.53909634, 0.64576635, 0.5567391 , 0.58053752, 0.62027959,
       0.58963584, 0.57650517, 0.54338321, 0.47379279, 0.51183193,
       0.32531788, 0.34732242, 0.34637054, 0.40038377, 0.31549223,
       0.51544218, 0.50169327, 0.5584698 , 0.52955372, 0.60191244,
       0.56634079, 0.46664279, 0.42165154, 0.52468504, 0.48364058,
       0.4962674 , 0.58743882, 0.58804758, 0.50929589, 0.56041038,
       0.59920481, 0.5877017 , 0.56224073, 0.48834456, 0.43285381,
       0.44442784, 0.43545671, 0.48664782, 0.48394735, 0.48655733,
       0.47696318, 0.5391098 , 0.53033079, 0.53840278, 0.53772892,
       0.57592237, 0.43682293, 0.42950422, 0.47313473, 0.52281024,
       0.46051563, 0.52008092, 0.44086296, 0.42220171, 0.3754415 ,
       0.38804769, 0.45031257, 0.52438531, 0.46510887, 0.36765635,
       0.35795024, 0.32350316, 0.26971465, 0.34035633, 0.46978

In [42]:


clf = RandomForestClassifier(max_depth=5, 
                             random_state=0, 
                             n_estimators=100,
                             class_weight="balanced"
                             #class_weight="[{0: 1, 1: 1}, {0: 1, 1: 1}, {0: 1, 1: 0}]"
                            )
clf.fit(x_train_2,y_train_2)
predict = clf.predict(x_test_2)

In [43]:
predict

array(['001', '010', '001', '001', '010', '001', '010', '010', '010',
       '010', '100', '001', '001', '001', '001', '100', '100', '001',
       '001', '001', '001', '001', '001', '001', '001', '001', '001',
       '001', '001', '001', '001', '001', '001', '001', '001', '001',
       '001', '010', '010', '010', '010', '001', '010', '010', '010',
       '010', '010', '100', '100', '100', '001', '001', '010', '001',
       '010', '010', '001', '001', '001', '100', '100', '100', '100',
       '001', '001', '001', '001', '001', '001', '001', '001', '001',
       '001', '001', '001', '001', '010', '010', '001', '001', '001',
       '001', '001'], dtype='<U3')

In [44]:
def svc(x_train, y_train, x_test, y_test):
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4, 0.06, 'scale'],
                     'C': [1, 10, 15, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000, 10000]}]
    scores = ['precision', 'recall']
    
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(
            SVC(max_iter=10000, random_state=5, class_weight="balanced", decision_function_shape='ovo'), 
            tuned_parameters, 
            scoring='%s_macro' % score,
            cv = 5
        )
        clf.fit(x_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

        print("Detailed classification report:")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        y_true, y_pred = y_test, clf.predict(x_test)
        print(classification_report(y_true, y_pred))
        print(y_pred)
        return y_true, y_pred

In [45]:
y_true, y_pred = svc(x_train, y_train_2, x_test, y_test_2)

# Tuning hyper-parameters for precision



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedM

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with Standar

Best parameters set found on development set:

{'C': 100, 'gamma': 0.06, 'kernel': 'rbf'}

Grid scores on development set:

0.118 (+/-0.096) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.118 (+/-0.096) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.284 (+/-0.326) for {'C': 1, 'gamma': 0.06, 'kernel': 'rbf'}
0.394 (+/-0.030) for {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}
0.213 (+/-0.004) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.213 (+/-0.004) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.399 (+/-0.036) for {'C': 10, 'gamma': 0.06, 'kernel': 'rbf'}
0.371 (+/-0.044) for {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
0.118 (+/-0.096) for {'C': 15, 'gamma': 0.001, 'kernel': 'rbf'}
0.118 (+/-0.096) for {'C': 15, 'gamma': 0.0001, 'kernel': 'rbf'}
0.404 (+/-0.021) for {'C': 15, 'gamma': 0.06, 'kernel': 'rbf'}
0.357 (+/-0.021) for {'C': 15, 'gamma': 'scale', 'kernel': 'rbf'}
0.401 (+/-0.230) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.178 (+/-0.054) for {'C': 100, 'gamma': 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:246: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with Standar

In [46]:
#data_eurusd = pd.read_csv("../EURUSD1m.csv")
data_mock = pd.read_csv("../MockData.csv")

In [47]:
data_eurusd.head()

NameError: name 'data_eurusd' is not defined

In [ ]:
data_lstm = features.split(dataset, 'lstm', settings['ensemble']['models'][1]['lstm'])

In [ ]:
os.chdir("..")
import Models